In [ ]:
%load_ext autoreload
%autoreload 1
%aimport dlqmc.nn, dlqmc.sampling, dlqmc.utils
%config InlineBackend.figure_format = 'svg' 
%config InlineBackend.print_figure_kwargs = \
    {'bbox_inches': 'tight', 'dpi': 300}

In [ ]:
import ipywidgets

import numpy as np
from scipy import special
import scipy.stats as sps
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, RandomSampler
from torch.distributions import Normal
from pyscf import gto, scf, dft
import pyscf
from pyscf.data.nist import BOHR
import time
from functools import partial
from tqdm.auto import tqdm, trange

from dlqmc.gto import *
from dlqmc.nn import *
from dlqmc.sampling import langevin_monte_carlo, hmc ,samples_from, metropolis
from dlqmc.fit import loss_local_energy
from dlqmc.utils import (
    plot_func, get_flat_mesh, assign_where, plot_func_xy,
    plot_func_x, integrate_on_mesh, assign_where
)
from dlqmc.physics import (
    local_energy, grad, quantum_force,nuclear_potential,
    nuclear_energy, laplacian, electronic_potential
)
from dlqmc.geom import *
from dlqmc.analysis import autocorr_coeff, blocking
from dlqmc.nn import ssp

### Activation function

In [ ]:
x=np.linspace(-5,5,100)
def relu(x):
    return x*[x>0]
plt.figure(figsize=(5,3))
plt.plot(x,relu(x).flatten(),color='grey',ls=':',label='relu')
#plt.plot(x,ssp(torch.from_numpy(x)).numpy()+np.log(2),color='k',label='shifted softplus')
plt.plot(x,soft(x),color='k',label='shifted softplus')
plt.legend(loc='upper left')
plt.xticks([])
plt.yticks([])
plt.xlabel("in")
plt.ylabel("out")
plt.show()

In [ ]:
def soft(x):
    return np.log(1+np.exp(x))

## $H_2^+$

In [ ]:
d_ref_h2p=1.9972 
h2p = Geometry([[-d_ref_h2p/2, 0., 0.], [d_ref_h2p/2, 0., 0.]], [1., 1.])
print(h2p)

In [ ]:
D = np.linspace(0.5,2,30)
E = []
for d in D:
    mol = gto.M(
        atom=[
            ['H', (-d, 0, 0)],
            ['H', (d, 0, 0)]
        ],
        unit='bohr',
        basis='4-31G',
        charge=1,
        spin=1,
    )
    mf = scf.RHF(mol)
    E.append(mf.kernel())
    
mol = gto.M(
        atom=[
            ['H', (-d_ref_h2p/2, 0, 0)],
            ['H', (d_ref_h2p/2, 0, 0)]
        ],
        unit='bohr',
        basis="cc-pV5Z",#'4-31G',
        charge=1,
        spin=1,
    )
mf = scf.RHF(mol)
print("E_min = "+str(mf.kernel()))
gtowf = PyscfGTOSlaterWF(mf)

In [ ]:
plt.plot(2*D,E)
plt.xlabel("distance nuclei in $a_0$" )
plt.ylabel("energy in $E_h$ ")
plt.title("Ground state energy of $H_2^+$ with respect to distance of nuclei")
plt.show()

In [ ]:
n_electrons=1
molecule = h2p

net = WFNet(molecule,n_electrons,ion_pot=0.7).cuda()


L = []
V = []

x_line = torch.cat((torch.linspace(-3, 3, 500)[:, None], torch.zeros((500, 3*n_electrons-1))), dim=1)
x_line=x_line.view(-1,n_electrons,3).cuda()
#mesh = get_3d_cube_mesh([(-6, 6), (-4, 4), (-4, 4)], [600, 400, 400])

opt = torch.optim.Adam(net.parameters(), lr=1e-2)
t_start=time.time()
scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=1, gamma=0.9995)

steps = 500 #5_000
batchsize = 2_000
n_resamplings = 5
n_walker = 1_000

sampler = langevin_monte_carlo(
    net,
    torch.randn(n_walker, n_electrons, 3, device='cuda'),
    tau=0.1,
)


#temporary
molecule._coords=molecule._coords.cuda()
molecule._charges=molecule._charges.cuda()

for i_step in range(steps):
        
    if i_step%(steps//4) == 0 or i_step==steps:
        with torch.no_grad():
            Psi2 = net(x_line)**2
            plt.plot(x_line[:,0 , 0].cpu().detach().numpy(), Psi2.cpu().detach().numpy(),label=i_step)
            #plt.show()
    
    scheduler.step()
    if i_step%(steps//n_resamplings)==0:
        print("resample                                                                        ",end="\r")
        rs,rs_psis  = samples_from(sampler,range(int(batchsize*steps/(n_resamplings*n_walker))))[0:-1]
        rs = rs.flatten(end_dim=1).cuda()
        rs_psis = rs_psis.flatten(end_dim=1).cuda()
        idx = torch.randperm(len(rs))
        rs = rs[idx]
        rs_psis = rs_psis[idx]
        
    r=rs[i_step%(steps//n_resamplings)*batchsize:(i_step%(steps//n_resamplings)+1)*batchsize]
    
    #insert some kinde of permutation to avoid training on some dependent samples in a batch
        
    E_loc,psi = local_energy(r,net,molecule,create_graph=True)

    wheigts=psi**2/rs_psis[i_step%(steps//n_resamplings)*batchsize:(i_step%(steps//n_resamplings)+1)*batchsize]**2
    
    if i_step<steps//10:
        loss = loss_local_energy(E_loc,wheigts,-1)
    else:
        loss = loss_local_energy(E_loc,wheigts,None)
            
    print("Progress {:2.0%}".format(i_step /steps)+"   ->"+"I"*(int(i_step/steps*100)//10)+"i"*(int(i_step/steps*100)%10)+"  "+"current loss = "+str(np.round(loss.item(),4))+"        ", end="\r")


    loss.backward()
    L.append(loss.cpu().detach().numpy())
    V.append(((E_loc**2-E_loc.mean()**2).mean()).cpu().detach().numpy())
    
    #torch.nn.utils.clip_grad_norm_(net.parameters(),1000)
    
    opt.step()
    opt.zero_grad()
    
plt.legend()
print("it took ="+str(np.round(time.time()-t_start,5))+"                    ")
    


In [ ]:
def normplot(x,y,norm,*args,**kwargs):
    if norm:
        plt.plot(x,y/np.max(np.abs(y)),*args,**kwargs)
    else:
        plt.plot(x,y,*args,**kwargs)

x_line = torch.cat((torch.linspace(-2, 2, 5000)[:, None], torch.zeros((5000, 3*n_electrons-1))), dim=1)
x_line=x_line.view(-1,n_electrons,3).cuda()
x_line.requires_grad = True
net.cuda()
f_line = net._featurize(x_line)
normed=True
normplot(x_line[:,0 , 0].cpu().detach().numpy(), torch.exp(net.deep_lin(f_line[0])).squeeze().cpu().detach().numpy(),label="sym",norm=normed)
normplot(x_line[:,0 , 0].cpu().detach().numpy(), net.nuc_asymp(f_line[1][0]).cpu().detach().numpy(),label="asym",norm=normed)
N = net.nuc_asymp(f_line[1][0]).cpu().detach().numpy()
normplot(x_line[:,0 , 0].cpu().detach().numpy(), net(x_line).cpu().detach().numpy(),label="WF",norm=normed,lw=2,color='k')
normplot(x_line[:,0 , 0].cpu().detach().numpy(),gtowf(x_line.detach().cpu()).numpy(),label="HFGTOWF",norm=normed)

plt.axhline(0,ls=':',color='k')
plt.axvline(0,ls=':',color='k')
plt.ylabel("wavefunction in arbitrary units")
plt.xlabel("position in $a_0$")
#plt.ylim(-1,2)
x_line.requires_grad = False
plt.legend()
#plt.savefig('lastrunwf.png')
plt.show()
plt.subplot2grid((2,1),(0,0))
plt.plot(L[:steps//10])
plt.yscale('log')
plt.subplot2grid((2,1),(1,0))
plt.plot(L[steps//10:])
plt.yscale('log')
#plt.savefig('lastrunloss.png')


In [ ]:
plt.plot(
    x_line[:, 0, 0].detach().cpu().numpy(),
    local_energy(x_line,lambda x: net(x), molecule)[0].cpu().detach().numpy()#*net(x_line).cpu().detach().numpy()**2
)
plt.ylim((-10, 10));

In [ ]:
if True:
    sampler = hmc(
        net,
        torch.randn(1000, n_electrons, 3, device='cuda'),
        dysteps=3,
        stepsize=0.2,
        tau = 0.1,
        cutoff = 1.0
    )
else:
    sampler = langevin_monte_carlo(
        net,
        torch.randn(1000, n_electrons, 3, device='cuda'),
        tau=0.1,
    )
t=time.time()
samples = samples_from(sampler,range(1000))[0].flatten(end_dim=1)
print("it took: "+str(time.time()-t))

In [ ]:
plt.hist2d(
    samples[:,0, 0].cpu().detach().numpy(),
    samples[:,0, 1].cpu().detach().numpy(),
    bins=100,
    range=[[-3, 3], [-3, 3]],
)                                   
plt.gca().set_aspect(1)

In [ ]:
net = net.cpu()
samples = samples.cpu()

In [ ]:
E_loc = local_energy(samples.view([-1,n_electrons,3]), lambda x: net(x),net.geom)[0]

In [ ]:
mean=E_loc.mean().item()

h = plt.hist(E_loc.detach().clamp(-1.5, 1).cpu().numpy(), bins=100,alpha = 0.5,color='b')
#plt.annotate("mean = "+str(np.round(mean,4)),(0,np.max(h[0])/2),color='b')
#plt.annotate("var     = "+str(np.round(np.var(E_loc.detach().numpy()),4)),(0,np.max(h[0])/2-np.max(h[0])/15),color='b')

#plt.savefig('lastruneloc.png')
plt.show()

## $H_2$ singlet

In [ ]:
D = np.linspace(0.2,2.2,30)
E = []
for d in D:
    mol = gto.M(
        atom=[
            ['H', (-d, 0, 0)],
            ['H', (d, 0, 0)]
        ],
        unit='bohr',
        basis='4-31G',
        charge=0,
        spin=0,
    )
    mf = scf.RHF(mol)
    E.append(mf.kernel())

In [ ]:
plt.plot(2*D,E)
plt.xlabel("distance nuclei in $a_0$" )
plt.ylabel("energy in $E_h$ ")
plt.title("Ground state energy of $H_2$ with respect to distance of nuclei")
plt.show()

In [ ]:
d_ref_h2=1.4
h2 = Geometry([[-d_ref_h2/2, 0., 0.], [d_ref_h2/2, 0., 0.]], [1., 1.])
print(h2)

In [ ]:
n_electrons=2
molecule = h2

net = WFNet(molecule,n_electrons,ion_pot=0.7).cuda()


L = []
V = []

x_line = torch.cat((torch.linspace(-3, 3, 500)[:, None], torch.zeros((500, 3*n_electrons-1))), dim=1)
x_line=x_line.view(-1,n_electrons,3).cuda()
#mesh = get_3d_cube_mesh([(-6, 6), (-4, 4), (-4, 4)], [600, 400, 400])

opt = torch.optim.Adam(net.parameters(), lr=1e-2)
t_start=time.time()
scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=1, gamma=0.9995)

steps = 5_000
batchsize = 2_000
n_resamplings = 25
n_walker = 1_000

sampler = langevin_monte_carlo(
    net,
    torch.randn(n_walker, n_electrons, 3, device='cuda'),
    tau=0.1,
)


#temporary
molecule._coords=molecule._coords.cuda()
molecule._charges=molecule._charges.cuda()

for i_step in range(steps):
        
    if i_step%(steps//4) == 0 or i_step==steps:
        with torch.no_grad():
            Psi2 = net(x_line)**2
            plt.plot(x_line[:,0 , 0].cpu().detach().numpy(), Psi2.cpu().detach().numpy(),label=i_step)
            #plt.show()
    
    scheduler.step()
    if i_step%(steps//n_resamplings)==0:
        print("resample                                                                        ",end="\r")
        rs,rs_psis  = samples_from(sampler,range(int(batchsize*steps/(n_resamplings*n_walker))))[0:-1]
        rs = rs.flatten(end_dim=1).cuda()
        rs_psis = rs_psis.flatten(end_dim=1).cuda()
        idx = torch.randperm(len(rs))
        rs = rs[idx]
        rs_psis = rs_psis[idx]
        
    r=rs[i_step%(steps//n_resamplings)*batchsize:(i_step%(steps//n_resamplings)+1)*batchsize]
    
    #insert some kinde of permutation to avoid training on some dependent samples in a batch
        
    E_loc,psi = local_energy(r,net,molecule,create_graph=True)

    wheigts=psi**2/rs_psis[i_step%(steps//n_resamplings)*batchsize:(i_step%(steps//n_resamplings)+1)*batchsize]**2
    
    if i_step<steps//10:
        loss = loss_local_energy(E_loc,wheigts,-1)
    else:
        loss = loss_local_energy(E_loc,wheigts,None)
            
    print("Progress {:2.0%}".format(i_step /steps)+"   ->"+"I"*(int(i_step/steps*100)//10)+"i"*(int(i_step/steps*100)%10)+"  "+"current loss = "+str(np.round(loss.item(),4))+"        ", end="\r")


    loss.backward()
    L.append(loss.cpu().detach().numpy())
    V.append(((E_loc**2-E_loc.mean()**2).mean()).cpu().detach().numpy())
    
    #torch.nn.utils.clip_grad_norm_(net.parameters(),1000)
    
    opt.step()
    opt.zero_grad()
    
plt.legend()
print("it took ="+str(np.round(time.time()-t_start,5))+"                    ")
    


In [ ]:
def normplot(x,y,norm,*args,**kwargs):
    if norm:
        plt.plot(x,y/np.max(np.abs(y)),*args,**kwargs)
    else:
        plt.plot(x,y,*args,**kwargs)

x_line = torch.cat((torch.linspace(-2, 2, 15000)[:, None], torch.zeros((15000, 3*n_electrons-1))), dim=1)
x_line=x_line.view(-1,n_electrons,3).cuda()
x_line.requires_grad = True
net.cuda()
f_line = net._featurize(x_line)
normed=True
normplot(x_line[:,0 , 0].cpu().detach().numpy(), torch.exp(net.deep_lin(f_line[0])).squeeze().cpu().detach().numpy(),label="sym",norm=normed)
normplot(x_line[:,0 , 0].cpu().detach().numpy(), net.nuc_asymp(f_line[1][0]).cpu().detach().numpy(),label="asym",norm=normed)
N = net.nuc_asymp(f_line[1][0]).cpu().detach().numpy()
normplot(x_line[:,0 , 0].cpu().detach().numpy(), net(x_line).cpu().detach().numpy(),label="WF",norm=normed,lw=2,color='k')

plt.axhline(0,ls=':',color='k')
plt.axvline(0,ls=':',color='k')

#plt.ylim(-1,2)
x_line.requires_grad = False
plt.legend()
#plt.savefig('lastrunwf.png')
plt.show()
plt.subplot2grid((2,1),(0,0))
plt.plot(L[:steps//10])
plt.yscale('log')
plt.subplot2grid((2,1),(1,0))
plt.plot(L[steps//10:])
plt.yscale('log')
#plt.savefig('lastrunloss.png')


In [ ]:
plt.plot(
    x_line[:, 0, 0].detach().cpu().numpy(),
    local_energy(x_line,lambda x: net(x), molecule)[0].cpu().detach().numpy()#*net(x_line).cpu().detach().numpy()**2
)
plt.ylim((-10, 10));

In [ ]:
if True:
    sampler = hmc(
        net,
        torch.randn(1000, n_electrons, 3, device='cuda'),
        dysteps=3,
        stepsize=0.2,
        tau = 0.1,
        cutoff = 1.0
    )
else:
    sampler = langevin_monte_carlo(
        net,
        torch.randn(1000, n_electrons, 3, device='cuda'),
        tau=0.1,
    )
t=time.time()
samples = samples_from(sampler,range(1000))[0].flatten(end_dim=1)
print("it took: "+str(time.time()-t))

In [ ]:
plt.hist2d(
    samples[:,0, 0].cpu().detach().numpy(),
    samples[:,0, 1].cpu().detach().numpy(),
    bins=100,
    range=[[-3, 3], [-3, 3]],
)                                   
plt.gca().set_aspect(1)

In [ ]:
net = net.cpu()
samples = samples.cpu()

In [ ]:
E_loc = local_energy(samples.view([-1,n_electrons,3]), lambda x: net(x),net.geom)[0]

In [ ]:
mean=E_loc.mean().item()

h = plt.hist(E_loc.detach().clamp(-1.5, 1).cpu().numpy(), bins=100,alpha = 0.5,color='b')
plt.annotate("mean = "+str(np.round(mean,4)),(0,np.max(h[0])/2),color='b')
plt.annotate("var     = "+str(np.round(np.var(E_loc.detach().numpy()),4)),(0,np.max(h[0])/2-np.max(h[0])/15),color='b')

#plt.savefig('lastruneloc.png')
plt.show()
print(mean)

## $H_2$ triplet